In [1]:
# !pip install google-cloud-aiplatform==1.36.3 litellm==0.14.1 trulens_eval==0.17.0 langchain==0.0.323 openai==0.27.6 chromadb==0.4.0 pypdf

In [3]:
import os
import utils

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "<GOOGLE_APPLICATION_CREDENTIALS>"
PROJECT_ID = "<PROJECT_ID>"
REGION = 'us-central1'

import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

## Loading Document

In [4]:
# Ingest PDF files
from langchain.document_loaders import PyPDFLoader

# Load GOOG's 10K annual report (92 pages).
file = "./20230203_alphabet_10K.pdf"
loader = PyPDFLoader(file)
documents = loader.load()

In [5]:
# split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(f"# of documents = {len(docs)}")

# of documents = 263


## Embeddings

In [6]:
from utils import CustomVertexAIEmbeddings

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

## Vector Store

In [7]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)


Waiting
....................................

## Retriever

In [8]:
# Expose index to the retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

## LLM

In [9]:
from langchain.llms import VertexAI

# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

## Trulens Evaluation

### Evaluation Questions

In [10]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What is the company's total revenue, and how has it changed over the past few years?
What are the main sources of revenue for the company?
How has the company's profitability (net income) trended over time?
What are the major components of the company's cost of goods sold (COGS)?
How have operating expenses (selling, general and administrative expenses) changed over the years?
What is the company's operating cash flow, and how does it compare to net income?
How does the company manage its working capital?
What is the overall financial health of the company's balance sheet?
How much debt does the company have, and what is its debt-to-equity ratio?
What significant investments or capital expenditures has the company made recently?
How does the company finance its capital investments?
What are the key risk factors mentioned in the 10-K filing?
How does the company assess and manage these risks?
What insights does the Management Discussion and Analysis section provide regarding the company

## Reset Database

In [27]:
from trulens_eval import (
    Tru,
    TruChain,
    Feedback,
    LiteLLM,
    feedback,
    Select,
)

tru = Tru()

tru.reset_database()

Deleted 4 rows.


### Triad Feedbacks

In [28]:
import numpy as np
# Initialize LiteLLM-based feedback function collection class:
# llm = OpenAI()
chat = LiteLLM(model_engine="chat-bison", verbose=True)
# Define a relevance function using LiteLLM
qa_relevance = (
    Feedback(chat.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)
# By default this will check relevance on the main app input and main app
# output.

qs_relevance = (
    Feedback(chat.relevance_with_cot_reasons, name="Context Relevance")
    .on_input()
    .on(Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content)
    .aggregate(np.mean)
)

# Groundedness between each context chunk and the response.
grounded = feedback.Groundedness(chat)
groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, 
                      name="Groundedness")
    .on(Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content .
✅ In Groundedness, input source will be set to __record__.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Chain Types

- `stuff`
- `map_reduce`
- `refine`
- `map_rerank`

In [ ]:
def text_format(text):
    return text.replace('_', '-').capitalize()

## QA Stuff Chain

In [29]:
from langchain.chains import RetrievalQA

chain_type = 'stuff'

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type=chain_type, 
    retriever=retriever, 
    return_source_documents=True
)

In [30]:
app_id=f'Retrieval {text_format(chain_type)} Chain'

tru_recorder = TruChain(
    qa,
    app_id=app_id,
    feedbacks=[qa_relevance, groundedness, qs_relevance]
)

✅ added app Retrieval Stuff Chain
✅ added feedback definition feedback_definition_hash_9db3c5dbfc9dfd237016008d518bf9c7
✅ added feedback definition feedback_definition_hash_c54a29c9b195a2705ba418ad8dd3fdff
✅ added feedback definition feedback_definition_hash_6d5cf3e928090fd959e958f8bbdb68a9


In [31]:
import time

with tru_recorder as recording:
    for question in eval_questions:
        response = qa({"query": question})
        time.sleep(20)

✅ added record record_hash_8e46ae5fe2b5e2bf05a0104c91187b58
✅ feedback result Answer Relevance DONE feedback_result_hash_e48045f0ff7dd483b75f766e229ed401
✅ feedback result Groundedness DONE feedback_result_hash_5f717beb448247df23a34864e45f06d3
✅ feedback result Context Relevance DONE feedback_result_hash_9543a13aa1cc3d54a4970e68e298a9b9
✅ added record record_hash_2b4b6edbd32269ee7fdb27dffedddde9
✅ feedback result Answer Relevance DONE feedback_result_hash_5a9e023a726ed320a0c15bc6757d119f
✅ feedback result Context Relevance DONE feedback_result_hash_6bc6a7856e5ecc2fef0877f27f41f823
✅ feedback result Groundedness DONE feedback_result_hash_1ea35a18f9f4432dff333b9b01e78a71
✅ added record record_hash_4d25641669d4715bce194b1218c55788
✅ feedback result Answer Relevance DONE feedback_result_hash_93f55b327bf4f53339161187fac01769
✅ feedback result Groundedness DONE feedback_result_hash_fb1935f0f32c0a6c667135536179f36a
✅ feedback result Context Relevance DONE feedback_result_hash_4ccc86c9cc305912

In [32]:
records, feedback = tru.get_records_and_feedback(app_ids=[app_id])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Retrieval Stuff Chain,"{""app_id"": ""Retrieval Stuff Chain"", ""tags"": ""-...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_8e46ae5fe2b5e2bf05a0104c91187b58,"""What is the company's total revenue, and how ...","""The company's total revenue is not mentioned ...",-,"{""record_id"": ""record_hash_8e46ae5fe2b5e2bf05a...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:02:21.371606"", ""...",2023-12-07T17:02:25.160800,0.0,0.0,0.35,[{'args': {'prompt': 'What is the company's to...,[{'args': {'source': 'In addition to the long-...,[{'args': {'prompt': 'What is the company's to...,3,0,0.0
1,Retrieval Stuff Chain,"{""app_id"": ""Retrieval Stuff Chain"", ""tags"": ""-...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_2b4b6edbd32269ee7fdb27dffedddde9,"""What are the main sources of revenue for the ...","""The main sources of revenue for the company a...",-,"{""record_id"": ""record_hash_2b4b6edbd32269ee7fd...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:02:45.625080"", ""...",2023-12-07T17:02:47.398560,0.9,1.0,0.00,[{'args': {'prompt': 'What are the main source...,"[{'args': {'source': 'Sources, Uses of Cash, a...",[{'args': {'prompt': 'What are the main source...,1,0,0.0
2,Retrieval Stuff Chain,"{""app_id"": ""Retrieval Stuff Chain"", ""tags"": ""-...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_4d25641669d4715bce194b1218c55788,"""How has the company's profitability (net inco...","""The company's profitability has trended upwar...",-,"{""record_id"": ""record_hash_4d25641669d4715bce1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:03:07.861523"", ""...",2023-12-07T17:03:09.640786,1.0,0.0,0.20,[{'args': {'prompt': 'How has the company's pr...,[{'args': {'source': 'ITEM 7. MANAGEMENT’S DIS...,[{'args': {'prompt': 'How has the company's pr...,1,0,0.0
3,Retrieval Stuff Chain,"{""app_id"": ""Retrieval Stuff Chain"", ""tags"": ""-...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_5d95a9202073f164a44a3183f765e61a,"""What are the major components of the company'...","""The question is not answerable. The context d...",-,"{""record_id"": ""record_hash_5d95a9202073f164a44...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:03:30.022034"", ""...",2023-12-07T17:03:32.003101,0.0,0.0,0.00,[{'args': {'prompt': 'What are the major compo...,[{'args': {'source': 'The main components of o...,[{'args': {'prompt': 'What are the major compo...,1,0,0.0
4,Retrieval Stuff Chain,"{""app_id"": ""Retrieval Stuff Chain"", ""tags"": ""-...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_9719309fde32b6e0c8d72f79f9107533,"""How have operating expenses (selling, general...","""Operating expenses increased by $11.4 billion...",-,"{""record_id"": ""record_hash_9719309fde32b6e0c8d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:03:52.520988"", ""...",2023-12-07T17:03:54.569605,1.0,0.8,0.85,[{'args': {'prompt': 'How have operating expen...,[{'args': {'source': 'Research and Development...,[{'args': {'prompt': 'How have operating expen...,2,0,0.0


In [34]:
# launches on http://localhost:8501/
tru.run_dashboard()

# tru.stop_dashboard() # stop if needed

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.102:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## QA Map-Reduce Chain

In [35]:
from langchain.chains import RetrievalQA

chain_type = "map_reduce"

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type=chain_type, 
    retriever=retriever, 
    return_source_documents=True
)

In [36]:
app_id=f'Retrieval {text_format(chain_type)} Chain'

tru_recorder = TruChain(
    qa,
    app_id=app_id,
    feedbacks=[qa_relevance, groundedness, qs_relevance]
)

✅ added app Retrieval Map-reduce Chain
✅ added feedback definition feedback_definition_hash_9db3c5dbfc9dfd237016008d518bf9c7
✅ added feedback definition feedback_definition_hash_c54a29c9b195a2705ba418ad8dd3fdff
✅ added feedback definition feedback_definition_hash_6d5cf3e928090fd959e958f8bbdb68a9


In [37]:
import time

with tru_recorder as recording:
    for question in eval_questions:
        response = qa({"query": question})
        time.sleep(20)

A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe1eb7fb40 is calling an instrumented method <function Chain.__call__ at 0x7efe54ee6f80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe1eb7fb40 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.schema.vectorstore.VectorStoreRetriever'> at 0x7efe1eb7f500 is calling an instrumented method <function VectorStoreRetriever._get_relevant_documents at 0x7efe900d75b0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7efe17b30f80) using this function.
A new object of type

✅ added record record_hash_b31b88a9d5b2bce56449af711bc0a271
✅ feedback result Answer Relevance DONE feedback_result_hash_6a09e422c9882e500310722565ec2210
✅ feedback result Groundedness DONE feedback_result_hash_2217d99703e0e29bd622450f7dbfbcee
✅ feedback result Context Relevance DONE feedback_result_hash_83be895afd1210fc1548494d08edf5f6


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe1eb7fb40 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain'> at 0x7efe1eb7c4c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1eb7eac0 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) usi

✅ added record record_hash_bcfd59b546125174cd2ac4f069297f1a
✅ feedback result Answer Relevance DONE feedback_result_hash_3f1306112842904e10bd63f384f3b913
✅ feedback result Groundedness DONE feedback_result_hash_b04b365e2756d7119251531f39af33ec
✅ feedback result Context Relevance DONE feedback_result_hash_ffaf1eb75404138b37ed1a81c0cd0359


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe1eb7fb40 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain'> at 0x7efe1eb7c4c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1eb7eac0 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) usi

...

✅ added record record_hash_da5edbbf98e228bf90fadad9860cc10e
✅ feedback result Answer Relevance DONE feedback_result_hash_d790d1a20b8e086ec64a6f12610dd64e
✅ feedback result Context Relevance DONE feedback_result_hash_a9db09c4e4b4758745a3056717bb51da
✅ feedback result Groundedness DONE feedback_result_hash_a0bbb7aafdb67d20afb928e6c0a86c71


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe1eb7fb40 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain'> at 0x7efe1eb7c4c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1eb7eac0 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) usi

✅ added record record_hash_defe20f50a6e2f2a605d775dfae7d9a9
✅ feedback result Answer Relevance DONE feedback_result_hash_e881d338b2617e49f4411c252024dccc
✅ feedback result Groundedness DONE feedback_result_hash_ea4e682c284136a031c9b5c6094564e3
✅ feedback result Context Relevance DONE feedback_result_hash_0df665c7db948dd766b1045f02a4f61c


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe1eb7fb40 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain'> at 0x7efe1eb7c4c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1eb7eac0 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) usi

✅ added record record_hash_b244038d0e3cde7f0a7d919719608aed
✅ feedback result Answer Relevance DONE feedback_result_hash_93d24b717c56737e86fa3edc052cc167
✅ feedback result Groundedness DONE feedback_result_hash_4b538b7e7786e5df7823400cb6da583d
✅ feedback result Context Relevance DONE feedback_result_hash_6005c5c0db1653901296daa9dc374609


In [38]:
records, feedback = tru.get_records_and_feedback(app_ids=[app_id])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Retrieval Map-reduce Chain,"{""app_id"": ""Retrieval Map-reduce Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_b31b88a9d5b2bce56449af711bc0a271,"""What is the company's total revenue, and how ...","""The company's total revenue is not mentioned ...",-,"{""record_id"": ""record_hash_b31b88a9d5b2bce5644...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:13:31.182183"", ""...",2023-12-07T17:13:39.382656,0.0,0.0,0.35,[{'args': {'prompt': 'What is the company's to...,[{'args': {'source': 'In addition to the long-...,[{'args': {'prompt': 'What is the company's to...,8,0,0.0
1,Retrieval Map-reduce Chain,"{""app_id"": ""Retrieval Map-reduce Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_bcfd59b546125174cd2ac4f069297f1a,"""What are the main sources of revenue for the ...","""The company does not mention the main sources...",-,"{""record_id"": ""record_hash_bcfd59b546125174cd2...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:13:59.913206"", ""...",2023-12-07T17:14:03.341114,0.0,0.0,0.00,[{'args': {'prompt': 'What are the main source...,"[{'args': {'source': 'Sources, Uses of Cash, a...",[{'args': {'prompt': 'What are the main source...,3,0,0.0
2,Retrieval Map-reduce Chain,"{""app_id"": ""Retrieval Map-reduce Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_98821abf72d51726962f74e3d6f353ae,"""How has the company's profitability (net inco...","""The company's profitability has trended upwar...",-,"{""record_id"": ""record_hash_98821abf72d51726962...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:14:23.853809"", ""...",2023-12-07T17:14:27.596242,0.8,0.0,0.20,[{'args': {'prompt': 'How has the company's pr...,[{'args': {'source': 'ITEM 7. MANAGEMENT’S DIS...,[{'args': {'prompt': 'How has the company's pr...,3,0,0.0
3,Retrieval Map-reduce Chain,"{""app_id"": ""Retrieval Map-reduce Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_45f453f804d883ad257df52729ae374f,"""What are the major components of the company'...","""The major components of our cost of goods sol...",-,"{""record_id"": ""record_hash_45f453f804d883ad257...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:14:48.152694"", ""...",2023-12-07T17:14:52.667579,1.0,0.0,0.00,[{'args': {'prompt': 'What are the major compo...,[{'args': {'source': 'The main components of o...,[{'args': {'prompt': 'What are the major compo...,4,0,0.0
4,Retrieval Map-reduce Chain,"{""app_id"": ""Retrieval Map-reduce Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_b289a6c045b94e7d60032cd01e20c570,"""How have operating expenses (selling, general...","""Operating expenses increased by $11.6 billion...",-,"{""record_id"": ""record_hash_b289a6c045b94e7d600...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:15:13.378380"", ""...",2023-12-07T17:15:17.476971,0.9,0.8,0.85,[{'args': {'prompt': 'How have operating expen...,[{'args': {'source': 'Research and Development...,[{'args': {'prompt': 'How have operating expen...,4,0,0.0


## QA Refine Chain

In [40]:
from langchain.chains import RetrievalQA

chain_type = "refine"

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type=chain_type, 
    retriever=retriever, 
    return_source_documents=True
)

In [41]:
app_id=f'Retrieval {text_format(chain_type)} Chain'

tru_recorder = TruChain(
    qa,
    app_id=app_id,
    feedbacks=[qa_relevance, groundedness, qs_relevance]
)

✅ added app Retrieval Refine Chain
✅ added feedback definition feedback_definition_hash_9db3c5dbfc9dfd237016008d518bf9c7
✅ added feedback definition feedback_definition_hash_c54a29c9b195a2705ba418ad8dd3fdff
✅ added feedback definition feedback_definition_hash_6d5cf3e928090fd959e958f8bbdb68a9


In [42]:
import time

with tru_recorder as recording:
    for question in eval_questions:
        response = qa({"query": question})
        time.sleep(20)

A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe173eca80 is calling an instrumented method <function Chain.__call__ at 0x7efe54ee6f80>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe173eca80 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.schema.vectorstore.VectorStoreRetriever'> at 0x7efe1724a480 is calling an instrumented method <function VectorStoreRetriever._get_relevant_documents at 0x7efe900d75b0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7efe17b30f80) using this function.
A new object of type

✅ added record record_hash_7089a26b8d01da26f7c0e106d8dffcaa
✅ feedback result Answer Relevance DONE feedback_result_hash_c7fbf76ac0dd475470669076bf210f46
✅ feedback result Groundedness DONE feedback_result_hash_c3caaaab093c16a7bb4951f9ff26fbb3
✅ feedback result Context Relevance DONE feedback_result_hash_1b4913af1efbd336bae343e3f78dfe8f


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe173eca80 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.refine.RefineDocumentsChain'> at 0x7efe1e428b00 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1724a600 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) using this

✅ added record record_hash_400cfb03adb768b9820013654cfb423f
✅ feedback result Answer Relevance DONE feedback_result_hash_53cf007978709f6971a4740504d1531d
✅ feedback result Context Relevance DONE feedback_result_hash_7b0ba4786ea3f840ccc466eeca889a79
✅ feedback result Groundedness DONE feedback_result_hash_f87186d066686e0b905fc4374eae798e


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe173eca80 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.refine.RefineDocumentsChain'> at 0x7efe1e428b00 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1724a600 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) using this

...

✅ added record record_hash_4c27af4f66e678eb5be13cf486e9734e
✅ feedback result Answer Relevance DONE feedback_result_hash_5f0cc029219320bea627e500ba0dcddc
✅ feedback result Context Relevance DONE feedback_result_hash_0fe3a8ca7fe01229249f4eb145523a0e
✅ feedback result Groundedness DONE feedback_result_hash_dcf88cfc1c96e304df912d6168f152d0


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe173eca80 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.refine.RefineDocumentsChain'> at 0x7efe1e428b00 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1724a600 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) using this

✅ added record record_hash_9805403445f24a24f4c7e8c1c4df817b
✅ feedback result Answer Relevance DONE feedback_result_hash_30f08a18a1142b330e51dfbf323fbf3b
✅ feedback result Context Relevance DONE feedback_result_hash_8b0813afd72976894f0b2cd763a86e19
✅ feedback result Groundedness DONE feedback_result_hash_2f52c8291f40c9de344faad7400ed273


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe173eca80 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.refine.RefineDocumentsChain'> at 0x7efe1e428b00 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
A new object of type <class 'langchain.chains.llm.LLMChain'> at 0x7efe1724a600 is calling an instrumented method <function LLMChain._call at 0x7efe54ee7d00>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain.llm_chain based on other object (0x7efe17b31a80) using this

✅ added record record_hash_abae9432856cca97ff8c2430179d94b6
✅ feedback result Answer Relevance DONE feedback_result_hash_ac587781328e3e322989435bc68199ce
✅ feedback result Context Relevance DONE feedback_result_hash_eca32d54954e31571d359fa8f7974d62
✅ feedback result Groundedness DONE feedback_result_hash_36398cc2fffb92a426dc32abeafe4f57


In [43]:
records, feedback = tru.get_records_and_feedback(app_ids=[app_id])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Retrieval Refine Chain,"{""app_id"": ""Retrieval Refine Chain"", ""tags"": ""...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_7089a26b8d01da26f7c0e106d8dffcaa,"""What is the company's total revenue, and how ...","""The company's total revenue is $136.8 billion...",-,"{""record_id"": ""record_hash_7089a26b8d01da26f7c...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:23:34.688177"", ""...",2023-12-07T17:23:37.830302,1.0,0.0,0.35,[{'args': {'prompt': 'What is the company's to...,[{'args': {'source': 'In addition to the long-...,[{'args': {'prompt': 'What is the company's to...,3,0,0.0
1,Retrieval Refine Chain,"{""app_id"": ""Retrieval Refine Chain"", ""tags"": ""...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_400cfb03adb768b9820013654cfb423f,"""What are the main sources of revenue for the ...","""The main sources of revenue for the company a...",-,"{""record_id"": ""record_hash_400cfb03adb768b9820...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:23:58.420494"", ""...",2023-12-07T17:24:03.484548,0.9,1.0,0.00,[{'args': {'prompt': 'What are the main source...,"[{'args': {'source': 'Sources, Uses of Cash, a...",[{'args': {'prompt': 'What are the main source...,5,0,0.0
2,Retrieval Refine Chain,"{""app_id"": ""Retrieval Refine Chain"", ""tags"": ""...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_21e215b059e863212760e3c5622e28de,"""How has the company's profitability (net inco...","""The company's profitability has trended upwar...",-,"{""record_id"": ""record_hash_21e215b059e86321276...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:24:24.090708"", ""...",2023-12-07T17:24:28.376944,1.0,0.0,0.20,[{'args': {'prompt': 'How has the company's pr...,[{'args': {'source': 'ITEM 7. MANAGEMENT’S DIS...,[{'args': {'prompt': 'How has the company's pr...,4,0,0.0
3,Retrieval Refine Chain,"{""app_id"": ""Retrieval Refine Chain"", ""tags"": ""...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_8ee20aeeb5a96ab3cf24fc35a8472c01,"""What are the major components of the company'...","""The major components of the company's cost of...",-,"{""record_id"": ""record_hash_8ee20aeeb5a96ab3cf2...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:24:48.954764"", ""...",2023-12-07T17:24:53.448979,0.8,0.0,0.00,[{'args': {'prompt': 'What are the major compo...,[{'args': {'source': 'The main components of o...,[{'args': {'prompt': 'What are the major compo...,4,0,0.0
4,Retrieval Refine Chain,"{""app_id"": ""Retrieval Refine Chain"", ""tags"": ""...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_9f45e6bf94595a7eab875c759d32a3a9,"""How have operating expenses (selling, general...","""Operating expenses (selling, general and admi...",-,"{""record_id"": ""record_hash_9f45e6bf94595a7eab8...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:25:13.991825"", ""...",2023-12-07T17:25:17.424905,1.0,0.5,0.90,[{'args': {'prompt': 'How have operating expen...,[{'args': {'source': 'Research and Development...,[{'args': {'prompt': 'How have operating expen...,3,0,0.0


## QA Map-rerank Chain

In [45]:
from langchain.chains import RetrievalQA

chain_type = "map_rerank"

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type=chain_type, 
    retriever=retriever, 
    return_source_documents=True
)

In [46]:
app_id=f'Retrieval {text_format(chain_type)} Chain'

tru_recorder = TruChain(
    qa,
    app_id=app_id,
    feedbacks=[qa_relevance, groundedness, qs_relevance]
)

✅ added app Retrieval Map-rerank Chain
✅ added feedback definition feedback_definition_hash_9db3c5dbfc9dfd237016008d518bf9c7
✅ added feedback definition feedback_definition_hash_c54a29c9b195a2705ba418ad8dd3fdff
✅ added feedback definition feedback_definition_hash_6d5cf3e928090fd959e958f8bbdb68a9


In [51]:
import time

with tru_recorder as recording:
    for question in eval_questions[11:]:
        response = qa({"query": question})
        time.sleep(20)

A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe17496600 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_rerank.MapRerankDocumentsChain'> at 0x7efe174954c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
/home/nt-nb/miniconda3/envs/trueval/lib/python3.10/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


✅ added record record_hash_11cec2ea8229166c1884bf0d16894b19
✅ feedback result Answer Relevance DONE feedback_result_hash_f0779ed63f130c18460ae5474bbc781e
✅ feedback result Groundedness DONE feedback_result_hash_a473a1657aa7de44375226cd43bb5531
✅ feedback result Context Relevance DONE feedback_result_hash_7d1314e9498054a3668a8a33a9782b61


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe17496600 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_rerank.MapRerankDocumentsChain'> at 0x7efe174954c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
/home/nt-nb/miniconda3/envs/trueval/lib/python3.10/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


✅ added record record_hash_71e278dddf4ca687a0c25b3b5c45ed30
✅ feedback result Answer Relevance DONE feedback_result_hash_ec3db1984043368951dbfc926118da86
✅ feedback result Groundedness DONE feedback_result_hash_e1db001603c013e0c97b9e9e37aeb123
✅ feedback result Context Relevance DONE feedback_result_hash_973a6757b6627fca5438d4e9247200bd


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe17496600 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_rerank.MapRerankDocumentsChain'> at 0x7efe174954c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
/home/nt-nb/miniconda3/envs/trueval/lib/python3.10/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


...

✅ added record record_hash_7322399d51d03e92f52ce9ff17641c82
✅ feedback result Answer Relevance DONE feedback_result_hash_afaf481ba7bfa602a039e1980b3aec77
✅ feedback result Context Relevance DONE feedback_result_hash_05b65b470a427ffae5c316de11e22479
✅ feedback result Groundedness DONE feedback_result_hash_ac75344ef3bd9ad4b21010e65567daff


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe17496600 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_rerank.MapRerankDocumentsChain'> at 0x7efe174954c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
/home/nt-nb/miniconda3/envs/trueval/lib/python3.10/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


✅ added record record_hash_3bb3ca782388352eba389de12fd426ff
✅ feedback result Answer Relevance DONE feedback_result_hash_0260bc7dd9e1edb9fa8a44cab5300e5e
✅ feedback result Groundedness DONE feedback_result_hash_f33ef875638ba71de478402281e97ca5
✅ feedback result Context Relevance DONE feedback_result_hash_4de02245b199ab6fcf2af14e477002bd


A new object of type <class 'langchain.chains.retrieval_qa.base.RetrievalQA'> at 0x7efe17496600 is calling an instrumented method <function BaseRetrievalQA._call at 0x7efe46264c10>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7efe17b336c0) using this function.
A new object of type <class 'langchain.chains.combine_documents.map_rerank.MapRerankDocumentsChain'> at 0x7efe174954c0 is calling an instrumented method <function BaseCombineDocumentsChain._call at 0x7efe81164a60>. The path of this call may be incorrect.
Guessing path of new object is app.combine_documents_chain based on other object (0x7efe17b30940) using this function.
/home/nt-nb/miniconda3/envs/trueval/lib/python3.10/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


✅ added record record_hash_34f0a272462c31cd733aeff3d14b0a2f
✅ feedback result Answer Relevance DONE feedback_result_hash_9f0033117a04c5d4e8a7daea7a3adf90
✅ feedback result Groundedness DONE feedback_result_hash_284ee90b1ef509a27ec0ce0b6621cca9
✅ feedback result Context Relevance DONE feedback_result_hash_6d3f7a30d5a1960fec33245d3bf2292e


In [48]:
records, feedback = tru.get_records_and_feedback(app_ids=[app_id])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Retrieval Map-rerank Chain,"{""app_id"": ""Retrieval Map-rerank Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_5fa3f5652c19f6b68984643704013f74,"""What is the company's total revenue, and how ...","""This document does not answer the question""",-,"{""record_id"": ""record_hash_5fa3f5652c19f6b6898...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:33:45.059390"", ""...",2023-12-07T17:33:47.247871,0.0,0.0,0.3,[{'args': {'prompt': 'What is the company's to...,[{'args': {'source': 'In addition to the long-...,[{'args': {'prompt': 'What is the company's to...,2,0,0.0
1,Retrieval Map-rerank Chain,"{""app_id"": ""Retrieval Map-rerank Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_c27f0358cd883ceb3f06f69660cf620f,"""What are the main sources of revenue for the ...","""This document does not answer the question""",-,"{""record_id"": ""record_hash_c27f0358cd883ceb3f0...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:34:07.701229"", ""...",2023-12-07T17:34:10.203245,0.0,0.0,0.0,[{'args': {'prompt': 'What are the main source...,"[{'args': {'source': 'Sources, Uses of Cash, a...",[{'args': {'prompt': 'What are the main source...,2,0,0.0
2,Retrieval Map-rerank Chain,"{""app_id"": ""Retrieval Map-rerank Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_f85a7eb70ab0154aaa0800abf94e96ba,"""How has the company's profitability (net inco...","""This document does not answer the question""",-,"{""record_id"": ""record_hash_f85a7eb70ab0154aaa0...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:34:30.805436"", ""...",2023-12-07T17:34:33.147119,0.0,0.0,0.2,[{'args': {'prompt': 'How has the company's pr...,[{'args': {'source': 'ITEM 7. MANAGEMENT’S DIS...,[{'args': {'prompt': 'How has the company's pr...,2,0,0.0
3,Retrieval Map-rerank Chain,"{""app_id"": ""Retrieval Map-rerank Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_f456c98a842f60750ab953a4489230c7,"""What are the major components of the company'...","""This document does not answer the question""",-,"{""record_id"": ""record_hash_f456c98a842f60750ab...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:34:53.604902"", ""...",2023-12-07T17:34:55.881363,0.0,0.0,0.0,[{'args': {'prompt': 'What are the major compo...,[{'args': {'source': 'The main components of o...,[{'args': {'prompt': 'What are the major compo...,2,0,0.0
4,Retrieval Map-rerank Chain,"{""app_id"": ""Retrieval Map-rerank Chain"", ""tags...",RetrievalQA(langchain.chains.retrieval_qa.base),record_hash_558ffbb7f1104f4c0051d53cd5e7a9c6,"""How have operating expenses (selling, general...","""Operating expenses increased by 10.6% from 20...",-,"{""record_id"": ""record_hash_558ffbb7f1104f4c005...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T17:35:16.424508"", ""...",2023-12-07T17:35:19.423937,1.0,1.0,0.9,[{'args': {'prompt': 'How have operating expen...,[{'args': {'source': 'Research and Development...,[{'args': {'prompt': 'How have operating expen...,2,0,0.0
